# Logistic Regression (scikit-learn)

In [1]:
import os

import joblib

import pandas as pd
import numpy as np

from sklearn import model_selection
from sklearn import linear_model
from sklearn import metrics

import warnings
warnings.filterwarnings('ignore')

## Data Loading

In [2]:
DATA_DIR = os.path.join("..", "data", "census")
DATA_FILE = "cleaned-census-data.csv"

In [3]:
# load pre-cleaned data from CSV file into pandas DataFrame
df = pd.read_csv(os.path.join(DATA_DIR, DATA_FILE), delimiter=',')

df.head()

age  capital-gain  capital-loss  hours-per-week  workclass_Local-gov  \
0   39          2174             0              40                    0   
1   50             0             0              13                    0   
2   38             0             0              40                    0   
3   53             0             0              40                    0   
4   28             0             0              40                    0   

   workclass_Private  workclass_Self-emp-inc  workclass_Self-emp-not-inc  \
0                  0                       0                           0   
1                  0                       0                           1   
2                  1                       0                           0   
3                  1                       0                           0   
4                  1                       0                           0   

   workclass_State-gov  workclass_Without-pay  ...   \
0                    1                      0  ...    
1                    0                      0  ...    
2                    0                      0  ...    
3                    0                      0  ...    
4                    0                      0  ...    

   occupation_Handlers-cleaners  occupation_Machine-op-inspct  \
0                             0                             0   
1                             0                             0   
2                             1                             0   
3                             1                             0   
4                             0                             0   

   occupation_Other-service  occupation_Priv-house-serv  \
0                         0                           0   
1                         0                           0   
2                         0                           0   
3                         0                           0   
4                         0                           0   

   occupation_Prof-specialty  occupation_Protective-serv  occupation_Sales  \
0                          0                           0                 0   
1                          0                           0                 0   
2                          0                           0                 0   
3                          0                           0                 0   
4                          1                           0                 0   

   occupation_Tech-support  occupation_Transport-moving  >50K  
0                        0                            0     0  
1                        0                            0     0  
2                        0                            0     0  
3                        0                            0     0  
4                        0                            0     0  

[5 rows x 44 columns]

In [4]:
# split into features and labels
features_df = df.drop('>50K', axis='columns')
labels_df = df['>50K']  # we are predicting whether an individual's income exceeds $50k/yr

## Data Splitting

In [5]:
# extract NumPy arrays from DataFrames
X = features_df.values
y = labels_df.values

# split data into training and testing sets
X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y, test_size=0.33)

# instantiate iterator that yields train/val indices for each fold of cross validation
validation_splitter = model_selection.KFold(n_splits=5, shuffle=True)

## Cross Validation

In [6]:
hyperparams = {
    'C': 1.0,
    'solver': 'lbfgs',
    'max_iter': 10000,
}

In [7]:
print(hyperparams, end=' ')

val_acc = 0  # track average validation accuracy across folds
for idxs_train, idxs_val in validation_splitter.split(X_train, y_train):
    # index into training data to produce train/val splits
    X_cross_train, y_cross_train = X[idxs_train], y[idxs_train]
    X_val, y_val = X[idxs_val], y[idxs_val]
    
    # create and fit model
    model = linear_model.LogisticRegression(**hyperparams)
    model.fit(X_cross_train, y_cross_train)
    
    # accumulate average validation accuracy
    val_acc += model.score(X_val, y_val)/validation_splitter.get_n_splits()

print(f"Validation accuracy: {val_acc}")

{'C': 1.0, 'solver': 'lbfgs', 'max_iter': 10000} Validation accuracy: 0.826048798656266


## Testing

In [8]:
# create and fit model using proven hyperparameters
model = linear_model.LogisticRegression(**hyperparams)
model.fit(X_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=10000, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='lbfgs',
          tol=0.0001, verbose=0, warm_start=False)

In [9]:
print(f"Training accuracy: {model.score(X_train, y_train)}")
print(f"Testing accuracy: {model.score(X_test, y_test)}")

Training accuracy: 0.8252058880747033
Testing accuracy: 0.8214074122835889


In [10]:
print(f"Training F-score: {metrics.f1_score(y_train, model.predict(X_train))}")
print(f"Testing F-score: {metrics.f1_score(y_test, model.predict(X_test))}")

Training F-score: 0.5815415664053404
Testing F-score: 0.5644565740889388


## Save Model

In [11]:
joblib.dump(model, os.path.join("..", "output", "logreg_basic.gz"))

['../output/logreg_basic.gz']